In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
# import seaborn as sns # En los boxplot de seaborn no se aprecian los outliers

df = pd.read_csv('df_todas_limpias.csv', sep = '|', encoding='utf-8')

# Imputació de 'IDEA_pc_1960', 'IDEA_pc_1960_69', 'IDEA_pc_1970_79', 'IDEA_pc_1980_89','IDEA_pc_1990_99', 'IDEA_pc_2000_10'

Primero tenemos que comprobar el principio _O todas o ninguna_

In [2]:
vars_zona = ['IDEA_pc_1960', 'IDEA_pc_1960_69', 'IDEA_pc_1970_79', 'IDEA_pc_1980_89','IDEA_pc_1990_99', 'IDEA_pc_2000_10']
nans_zona = df[vars_zona].isnull().sum(axis = 1)
print('Hay {} registros sin ningún Nan'.format(np.sum(nans_zona == 0)))
print('Hay {} registros con todo Nan'.format(np.sum(nans_zona == 6)))
print('Hay {} con mix'.format(nans_zona.shape[0]-np.sum(nans_zona == 0)-np.sum(nans_zona == 6)))

Hay 7234 registros sin ningún Nan
Hay 2700 registros con todo Nan
Hay 0 con mix


Como son Nans al mismo tiempo, el problema se simplifica

Primero tenemos que realizar dos groupby, uno por CP y el otro por provincias sobre los datos sin Nans

In [3]:
group_cp = df[['HY_cod_postal']+vars_zona].dropna().groupby('HY_cod_postal').mean()
group_provincia = df[['HY_provincia']+vars_zona].dropna().groupby('HY_provincia').mean()

In [4]:
print('Se han descartado {} códigos postales por ser todo Nans'.format(df['HY_cod_postal'].unique().shape[0]-group_cp.shape[0]))

Se han descartado 114 códigos postales por ser todo Nans


Por lo que para esos 114 casos emplearemos las provincias.

In [5]:
print('Se han descartado {} provincias por ser todo Nans'.format(df['HY_provincia'].unique().shape[0]-group_provincia.shape[0]))

Se han descartado 0 provincias por ser todo Nans


Por lo tanto, procedamos con la imputación por códigos postales.

Primero necesitamos obtener los códigos postales en los que hay Nans

In [6]:
# Aprovechando la propiedad de que las seis variables son Nans simultaneamente podemmos hacer lo siguiente para obtener
#      dichos códigos
codigos_nans = df.HY_cod_postal[df.IDEA_pc_1960.isnull()] # Valdría cualquiera de las 6 variables.

In [7]:
a = codigos_nans.unique().shape[0]
b = np.intersect1d(codigos_nans.unique(),group_cp.index).shape[0]
print('Dentro de los {} codigos de codigos_nans, hay {} códigos que conseguiremos completar y {} que no (El CP que no están registrados en group_cp)'.format(a,b,a-b))

Dentro de los 552 codigos de codigos_nans, hay 438 códigos que conseguiremos completar y 114 que no (El CP que no están registrados en group_cp)


Por lo que sabiendo que vamos a conseguir reducir el número de Nans podemos proceder.

In [8]:
# Como sabemos que códigos podremos completar y cuales no, solo utilizaremos los que se pueden completar
cods = np.intersect1d(codigos_nans.unique(),group_cp.index)
# Cuales son los índices de los Nans
index_nan = df.index[df.IDEA_pc_1960.isnull()]
for cod in cods:
    # Explicación del indexado: De todos los códigos que coinciden con el nuestro nos quedamos con los que tienen índice
    #      nan, y para poder acceder a df, necesitamos los índices de Nan que cumplen lo del código.
    i = index_nan[(df.HY_cod_postal == cod)[index_nan]]
    df.loc[i, vars_zona] = group_cp.loc[cod].values

In [9]:
nans_zona = df[vars_zona].isnull().sum(axis = 1)
print('Hay {} registros sin ningún Nan'.format(np.sum(nans_zona == 0)))
print('Hay {} registros con todo Nan'.format(np.sum(nans_zona == 6)))
print('Hay {} con mix'.format(nans_zona.shape[0]-np.sum(nans_zona == 0)-np.sum(nans_zona == 6)))

Hay 9756 registros sin ningún Nan
Hay 178 registros con todo Nan
Hay 0 con mix


Por lo que una vez imputados los valores por la media del código postal vamos a guardar _group\_cp_ para luego poder imputar en test

In [10]:
group_cp.to_csv('group_cp_fill_atiguedad_zonas.csv', sep = '|', encoding='utf-8')

Ahora obtendremos los CP de una manera similar a la anterior y procederemos por provincias

In [11]:
provincias_nans = df.HY_provincia[df.IDEA_pc_1960.isnull()] # Valdría cualquiera de las 6 variables.
# Como sabemos que códigos podremos completar y cuales no, solo utilizaremos los que se pueden completar
provincias = np.intersect1d(provincias_nans.unique(),group_provincia.index)
# Cuales son los índices de los Nans
index_nan = df.index[df.IDEA_pc_1960.isnull()]

for provincia in provincias:
    # Idem
    i = index_nan[(df.HY_provincia == provincia)[index_nan]]
    df.loc[i, vars_zona] = group_provincia.loc[provincia].values

In [12]:
nans_zona = df[vars_zona].isnull().sum(axis = 1)
print('Hay {} registros sin ningún Nan'.format(np.sum(nans_zona == 0)))
print('Hay {} registros con todo Nan'.format(np.sum(nans_zona == 6)))
print('Hay {} con mix'.format(nans_zona.shape[0]-np.sum(nans_zona == 0)-np.sum(nans_zona == 6)))

Hay 9934 registros sin ningún Nan
Hay 0 registros con todo Nan
Hay 0 con mix


Guardamos el grupo de provincias

In [13]:
group_provincia.to_csv('group_provincia_fill_atiguedad_zonas.csv', sep = '|', encoding='utf-8')

Y el DataFrame nuevo

In [14]:
df.to_csv('df_atiguedadIDEA_filled.csv', sep = '|', encoding='utf-8', index = False)

### Caso en el que aun siga habiendo Nans

Si continuaramos con Nans en nuestras observaciones, no tendríamos mas remedio que poner todas las variables a 1/6. (Si la cantidad de nans fuera significativa, lo que tendríamos que hacer es crear una nueva clase llamada _No\_consta_ o similares)

In [15]:
if np.sum(df.IDEA_pc_1960.isnull())>0:
    index_nan = df.index[df.IDEA_pc_1960.isnull()]
    df.loc[index_nan, vars_zona] = 1/6

# Generalizando:

Tomando como referencia el código anterior vamos a implementar varias funciones para poderlas aplicar en los siguientes grupos de variables venideros.

La idea es que las funciones nos ahorren código y nos faciliten la tarea de imputar valores faltantes.

**Nota:** Las funciones que diseñamos se supone que estan diseñadas para funcionar con toda la fila de Nans.

In [16]:
vars_imput = ['IDEA_pc_comercio','IDEA_pc_industria', 'IDEA_pc_oficina', 'IDEA_pc_otros','IDEA_pc_residencial', 'IDEA_pc_trast_parking']

In [17]:
def InfoNans(df, vars_imput):
    nans_zona = df[vars_imput].isnull().sum(axis = 1)
    print('Hay {} registros sin ningún Nan'.format(np.sum(nans_zona == 0)))
    print('Hay {} registros con todo Nan'.format(np.sum(nans_zona == len(vars_imput))))
    print('Hay {} con mix'.format(nans_zona.shape[0]-np.sum(nans_zona == 0)-np.sum(nans_zona == len(vars_imput))))

def InfoImputarNans(df, vars_imput):
    # Vemos cuantos Nans hay.
    InfoNans(df, vars_imput)
    
    # Obtenemos nuestros df
    group_cp = df[['HY_cod_postal']+vars_imput].dropna().groupby('HY_cod_postal').mean()
    group_provincia = df[['HY_provincia']+vars_imput].dropna().groupby('HY_provincia').mean()
    
    # Info útil
    print('Se han descartado {} códigos postales por ser todo Nans'.format(df['HY_cod_postal'].unique().shape[0]-group_cp.shape[0]))
    print('Se han descartado {} provincias por ser todo Nans'.format(df['HY_provincia'].unique().shape[0]-group_provincia.shape[0]))
    
    # Aprovechando la propiedad de que las seis variables son Nans simultaneamente podemmos hacer lo siguiente para obtener
    #      dichos códigos
    codigos_nans = df.HY_cod_postal[df[vars_imput[0]].isnull()] # Valdría cualquiera de las 6 variables.
    
    a = codigos_nans.unique().shape[0]
    b = np.intersect1d(codigos_nans.unique(),group_cp.index).shape[0]
    print('Dentro de los {} codigos de codigos_nans, hay {} códigos que conseguiremos completar y {} que no (Los CP que no están registrados en group_cp)'.format(a,b,a-b))

def ImputarNans_cp(df, vars_imput, var):
    '''
    df --> Nuestro dataframe a modificar
    vars_imput --> Variables que queremos imputar.
    var --> Variable por la que queremos realizar la agrupación (HY_cod_postal ó HY_provincia)
    '''
    # Obtenemos nuestros df de grupos
    group_cp = df[[var]+vars_imput].dropna().groupby(var).mean()
    
    # Obtenemos los CP que son Nans
    codigos_nans = df.loc[df[vars_imput[0]].isnull(), var] # Valdría cualquiera de las 6 variables.
    
    # Como sabemos que códigos podremos completar y cuales no, solo utilizaremos los que se pueden completar
    cods = np.intersect1d(codigos_nans.unique(),group_cp.index)
    # Cuales son los índices de los Nans
    index_nan = df.index[df[vars_imput[0]].isnull()]
    for cod in cods:
        # Explicación del indexado: De todos los códigos que coinciden con el nuestro nos quedamos con los que tienen índice
        #      nan, y para poder acceder a df, necesitamos los índices de Nan que cumplen lo del código.
        i = index_nan[(df[var] == cod)[index_nan]]
        df.loc[i, vars_imput] = group_cp.loc[cod].values
        
    # Vemos cuantos Nans hay.
    InfoNans(df, vars_imput)
    
    # Devolvemos los dataframes
    return df, group_cp

Primero comprobamos que se cumple el principio _o todas o ninguna_ con las filas de Nans

In [18]:
InfoNans(df, vars_imput)

Hay 7234 registros sin ningún Nan
Hay 2700 registros con todo Nan
Hay 0 con mix


Y una vez comprobado, ya podemos pasar a la imputación

In [19]:
df, group_cp = ImputarNans_cp(df, vars_imput, var = 'HY_cod_postal')
InfoNans(df, vars_imput)
print('-'*30)
df, group_provincia = ImputarNans_cp(df, vars_imput, var = 'HY_provincia')
InfoNans(df, vars_imput)

Hay 9756 registros sin ningún Nan
Hay 178 registros con todo Nan
Hay 0 con mix
Hay 9756 registros sin ningún Nan
Hay 178 registros con todo Nan
Hay 0 con mix
------------------------------
Hay 9934 registros sin ningún Nan
Hay 0 registros con todo Nan
Hay 0 con mix
Hay 9934 registros sin ningún Nan
Hay 0 registros con todo Nan
Hay 0 con mix


Guardamos los df de resultado

In [20]:
df.to_csv('./DF_grupos/df_filled_{}.csv'.format(vars_imput[0]), sep = '|', encoding='utf-8', index = False)
group_cp.to_csv('./DF_grupos/group_cp_{}.csv'.format(vars_imput[0]), sep = '|', encoding='utf-8')
group_provincia.to_csv('./DF_grupos/group_prov_{}.csv'.format(vars_imput[0]), sep = '|', encoding='utf-8')

# Resto de variables

Una vez demostrado que nuestras funciones son operativas, vamos a proceder a utilizarlas con los siguientes grupos de variables:
+ 'IDEA_ind_tienda', 'IDEA_ind_turismo', 'IDEA_ind_alimentacion'
+ 'IDEA_ind_riqueza'
+ 'IDEA_rent_alquiler'
+ 'IDEA_ind_elasticidad', 'IDEA_ind_liquidez'
+ 'IDEA_unitprice_sale_residential', 'IDEA_price_sale_residential', 'IDEA_stock_sale_residential'
+ 'IDEA_demand_sale_residential'
+ 'IDEA_unitprice_rent_residential', 'IDEA_price_rent_residential', 'IDEA_stock_rent_residential'
+ 'IDEA_demand_rent_residential'

Pero primero vamos a comprobar si cumplen la condición de todo nans:

In [21]:
# Lista con la lista de variables a tratar
var_list = [
    ['IDEA_ind_tienda', 'IDEA_ind_turismo', 'IDEA_ind_alimentacion'],
    ['IDEA_ind_riqueza'],
    ['IDEA_rent_alquiler'],
    ['IDEA_ind_elasticidad', 'IDEA_ind_liquidez'],
    ['IDEA_unitprice_sale_residential', 'IDEA_price_sale_residential', 'IDEA_stock_sale_residential'],
    ['IDEA_demand_sale_residential'],
    ['IDEA_unitprice_rent_residential', 'IDEA_price_rent_residential', 'IDEA_stock_rent_residential'],
    ['IDEA_demand_rent_residential']   
]

In [22]:
print('*'*50)
for vars_group in var_list:
    InfoNans(df, vars_group)
    print('*'*50)

**************************************************
Hay 7227 registros sin ningún Nan
Hay 2707 registros con todo Nan
Hay 0 con mix
**************************************************
Hay 7313 registros sin ningún Nan
Hay 2621 registros con todo Nan
Hay 0 con mix
**************************************************
Hay 6882 registros sin ningún Nan
Hay 3052 registros con todo Nan
Hay 0 con mix
**************************************************
Hay 4844 registros sin ningún Nan
Hay 5090 registros con todo Nan
Hay 0 con mix
**************************************************
Hay 7309 registros sin ningún Nan
Hay 2625 registros con todo Nan
Hay 0 con mix
**************************************************
Hay 7251 registros sin ningún Nan
Hay 2683 registros con todo Nan
Hay 0 con mix
**************************************************
Hay 6946 registros sin ningún Nan
Hay 2988 registros con todo Nan
Hay 0 con mix
**************************************************
Hay 7309 registros sin ningún Na

Por lo que podemos aplicar nuestras funciones.

In [23]:
for vars_group in var_list:
    print('*'*50)
    print('Variables:', vars_group)
    print('-'*10+' CP '+'-'*10)
    df, group_cp = ImputarNans_cp(df, vars_group, var = 'HY_cod_postal')
    InfoNans(df, vars_group)
    print('-'*10+' Provincias '+'-'*10)
    df, group_provincia = ImputarNans_cp(df, vars_group, var = 'HY_provincia')
    InfoNans(df, vars_group)
    
    df.to_csv('./DF_grupos/df_filled_{}.csv'.format(vars_group[0]), sep = '|', encoding='utf-8', index = False)
    group_cp.to_csv('./DF_grupos/group_cp_{}.csv'.format(vars_group[0]), sep = '|', encoding='utf-8')
    group_provincia.to_csv('./DF_grupos/group_prov_{}.csv'.format(vars_group[0]), sep = '|', encoding='utf-8')

**************************************************
Variables: ['IDEA_ind_tienda', 'IDEA_ind_turismo', 'IDEA_ind_alimentacion']
---------- CP ----------
Hay 9755 registros sin ningún Nan
Hay 179 registros con todo Nan
Hay 0 con mix
Hay 9755 registros sin ningún Nan
Hay 179 registros con todo Nan
Hay 0 con mix
---------- Provincias ----------
Hay 9934 registros sin ningún Nan
Hay 0 registros con todo Nan
Hay 0 con mix
Hay 9934 registros sin ningún Nan
Hay 0 registros con todo Nan
Hay 0 con mix
**************************************************
Variables: ['IDEA_ind_riqueza']
---------- CP ----------
Hay 9812 registros sin ningún Nan
Hay 122 registros con todo Nan
Hay 0 con mix
Hay 9812 registros sin ningún Nan
Hay 122 registros con todo Nan
Hay 0 con mix
---------- Provincias ----------
Hay 9934 registros sin ningún Nan
Hay 0 registros con todo Nan
Hay 0 con mix
Hay 9934 registros sin ningún Nan
Hay 0 registros con todo Nan
Hay 0 con mix
**************************************************

Comprobamos cuantos Nans hay en cada variable

In [24]:
df.isnull().sum(axis = 0)

HY_id                                 0
HY_cod_postal                         0
HY_provincia                          0
HY_descripcion                     3926
HY_distribucion                    6728
HY_tipo                               0
HY_antiguedad                      5463
HY_metros_utiles                      0
HY_metros_totales                     0
HY_num_banos                          0
HY_cert_energ                         0
HY_num_terrazas                       0
HY_ascensor                           0
HY_trastero                           0
HY_num_garajes                        0
HY_precio                             0
HY_precio_anterior                    0
IDEA_area                          2621
IDEA_poblacion                     2625
IDEA_densidad                      2625
IDEA_pc_1960                          0
IDEA_pc_1960_69                       0
IDEA_pc_1970_79                       0
IDEA_pc_1980_89                       0
IDEA_pc_1990_99                       0


Por lo que tenemos que las únicas variables que quedan con Nans son:
+ IDEA_area, IDEA_poblacion, IDEA_densidad $\longrightarrow$ Las completaremos con datos externos.
+ HY_descripcion, HY_distribucion $\longrightarrow$ Las eliminaremos ya que tenemos sus variables asociadas PV_longitud_descripcion, PV_longitud_distribucion
+ PV_ind_elasticidad $\longrightarrow$ La cambiaremos por su variante sn Nans
+ HY_antiguedad $\longrightarrow$ La intentaremos predecir mediante las variables de antigüedad de las zonas.


# Guardamos el dataframe final con todo arreglado

In [25]:
df.to_csv('df_filled_final.csv', sep = '|', encoding='utf-8', index = False)